In [24]:
import numpy as np
import sympy as sp
from sympy import symbols, sympify, Eq, diff
import os
import re
from m2_functions import *

def extract_variables(equation):
    # Use regex to find all variable-like patterns
    variables = re.findall(r'\b[a-zA-Z_][a-zA-Z0-9_]*\b', equation)
    return list(set(variables))

def get_variables_from_equations(equations):
    # Apply extract_variables to each equation in the list
    return [extract_variables(eq) for eq in equations]

def collect_variables_in_sequence_and_reverse(equations):
    """
    Collects variables from each equation in sequence and reverses the list.
    Variables from equation 1 will be at the rightmost side,
    variables from equation 2 will be to the left of those from equation 1, and so on.
    
    Parameters:
        equations (list): List of equations as strings.
    
    Returns:
        list: Reversed list of variables collected in sequence.
    """
    all_vars = []
    
    # Get variables from each equation in sequence
    equation_variables = get_variables_from_equations(equations)
    
    # Add variables in sequence, avoiding duplicates
    for vars_list in equation_variables:
        for var in vars_list:
            if var not in all_vars:
                all_vars.append(var)
    
    # Reverse the list so variables from equation 1 are at the rightmost side
    all_vars.reverse()
    
    return all_vars

def generate_dataset_from_grobner_basis(grobner_basis, variables, num_points=10000, seed=42, tol=1e-6, max_iter=100):
    np.random.seed(seed)
    # Initialize the data dictionary - but don't fill it with data yet
    data = {}
    
    sym_vars = symbols(' '.join(variables))
    
    # Sort Gröbner basis by complexity (number of variables)
    sorted_basis = sorted(grobner_basis, key=lambda eq: len(extract_variables(eq)))
    
    num_valid_points = num_points  # Track how many valid points we have
    
    for eq in sorted_basis:
        eq_expr = sympify(eq)
        eq_variables = extract_variables(eq)
        
        # Find variables in this equation that don't have data yet
        unknown_vars = [var for var in eq_variables if var not in data]
        
        # If all variables already have data, skip this equation
        if not unknown_vars:
            continue
        
        # Generate Gaussian data for all but one unknown variable
        for var in unknown_vars[:-1]:
            data[var] = np.random.normal(loc=0, scale=1, size=num_valid_points)
        
        # Solve for the last unknown variable
        last_var = unknown_vars[-1]
        last_var_sym = sym_vars[variables.index(last_var)]
        
        f = eq_expr
        f_prime = diff(f, last_var_sym)
        
        last_var_data = np.zeros(num_valid_points)
        valid_indices = []
        
        for i in range(num_valid_points):
            # Substitute values for known variables
            subs_dict = {sym_vars[variables.index(v)]: data[v][i] for v in data if v in eq_variables}
            
            f_subs = f.subs(subs_dict)
            f_prime_subs = f_prime.subs(subs_dict)
            
            # Newton-Raphson method to solve for the last variable
            x0 = 0.0
            x = x0
            for _ in range(max_iter):
                fx = f_subs.subs(last_var_sym, x)
                fpx = f_prime_subs.subs(last_var_sym, x)
                if fpx == 0:
                    break
                x_new = x - fx / fpx
                if abs(x_new - x) < tol:
                    x = x_new
                    break
                x = x_new
            
            # Check if the solution is valid
            if abs(f_subs.subs(last_var_sym, x)) < tol and (isinstance(x, (int, float)) or (hasattr(x, 'is_real') and x.is_real)):
                last_var_data[i] = float(x)
                valid_indices.append(i)
        
        # Filter out invalid data points
        if valid_indices:
            valid_indices = np.array(valid_indices)
            for var in data:
                data[var] = data[var][valid_indices]
            last_var_data = last_var_data[valid_indices]
            data[last_var] = last_var_data
            # Update the number of valid points
            num_valid_points = len(valid_indices)
        else:
            print(f"No valid solutions found for variable {last_var} in equation {eq}")
            # If no valid solutions, we might need to adjust our approach
            return None  # Or handle this case differently
    
    # Any variables not in the data dictionary yet should get random Gaussian values
    # But make sure to generate only as many points as we have after filtering
    if data:  # Check if we have any data
        current_size = len(next(iter(data.values())))
        for var in variables:
            if var not in data:
                data[var] = np.random.normal(loc=0, scale=1, size=current_size)
    else:
        # If we have no data at all, generate fresh random data for all variables
        for var in variables:
            data[var] = np.random.normal(loc=0, scale=1, size=num_points)
    
    return data

def generate_data_for_system(equations, num_points=10000, seed=42):
    """
    Generate data for a system of equations.

    Parameters:
        equations (list): List of equations as strings.
        num_points (int): Number of data points to generate.
        seed (int): Random seed for reproducibility.

    Returns:
        dict: A dictionary where keys are variable names and values are numpy arrays of data.
    """
    variables = collect_variables_in_sequence_and_reverse(equations)
    projection(variables, equations, variables, [], filename='temp_grobner.txt')

    with open('temp_grobner.txt', 'r') as file:
        content = file.read()
        grobner_basis = re.findall(r'Polynomials of the Grobner basis of the eliminated ideal:\s*(.*?)\n\n', content, re.DOTALL)
        grobner_basis = grobner_basis[0].split('\n') if grobner_basis else []

    os.remove('temp_grobner.txt')

    if not grobner_basis:
        raise ValueError("No Gröbner basis found in the projection output.")

    return generate_dataset_from_grobner_basis(grobner_basis, variables, num_points, seed)

In [26]:
def parse_data(file_path):
    with open(file_path, 'r') as file:
        content = file.read()

    # Split the content into individual data points
    data_points = content.split('\n\n')

    # Initialize lists to store the parsed data
    variables_list = []
    measured_variables_list = []
    constants_list = []
    non_measured_variables_list = []
    equations_list = []
    derivatives_list = []

    # Regular expressions to extract the relevant information
    variables_pattern = re.compile(r'Variables:\s*(.*?)\n', re.DOTALL)
    measured_variables_pattern = re.compile(r'Measured Variables:\s*(.*?)\n', re.DOTALL)
    constants_pattern = re.compile(r'Constants:\s*(.*?)\n', re.DOTALL)
    non_measured_variables_pattern = re.compile(r'Non Measured Variables:\s*(.*?)\n', re.DOTALL)
    equations_pattern = re.compile(r'Equations:\s*(.*?)(?=\n\s*\d+:|\Z)', re.DOTALL)
    derivatives_pattern = re.compile(r'Derivatives:\s*(.*?)\n', re.DOTALL)

    for data_point in data_points:
        # Extract variables
        variables_match = variables_pattern.search(data_point)
        if variables_match:
            variables = variables_match.group(1).strip().split(',')
            variables_list.append(variables)

        # Extract measured variables
        measured_variables_match = measured_variables_pattern.search(data_point)
        if measured_variables_match:
            measured_variables = measured_variables_match.group(1).strip().split(',')
            measured_variables_list.append(measured_variables)

        # Extract constants
        constants_match = constants_pattern.search(data_point)
        if constants_match:
            constants = constants_match.group(1).strip().split(',')
            constants_list.append(constants)

        # Extract non-measured variables
        non_measured_variables_match = non_measured_variables_pattern.search(data_point)
        if non_measured_variables_match:
            non_measured_variables = non_measured_variables_match.group(1).strip().split(',')
            non_measured_variables_list.append(non_measured_variables)

        # Extract equations
        equations_match = equations_pattern.search(data_point)
        if equations_match:
            equations = equations_match.group(1).strip().split('\n')
            equations = [eq.strip() for eq in equations if eq.strip()]  # Remove empty lines
            equations_list.append(equations)

        # Extract derivatives
        derivatives_match = derivatives_pattern.search(data_point)
        if derivatives_match:
            derivatives = derivatives_match.group(1).strip().split(',')
            derivatives_list.append(derivatives)

    return {
        'variables': variables_list,
        'measured_variables': measured_variables_list,
        'constants': constants_list,
        'non_measured_variables': non_measured_variables_list,
        'equations': equations_list,
        'derivatives': derivatives_list
    }

def sort_equations_by_variable_count(equations):
    """
    Sorts a list of equations from least to most number of variables.
    
    Parameters:
        equations (list): List of equations as strings.
    
    Returns:
        list: Sorted list of equations from least to most variables.
    """
    # Create a list of (equation, variable_count) tuples
    equation_variable_counts = []
    for eq in equations:
        variables = extract_variables(eq)
        equation_variable_counts.append((eq, len(variables)))
    
    # Sort the list based on variable count
    sorted_equations = [eq for eq, count in sorted(equation_variable_counts, key=lambda x: x[1])]
    
    return sorted_equations

In [27]:
# Path to the text file
file_path = 'synthetic_axioms.txt'

# Parse the data
parsed_data = parse_data(file_path)

equations = parsed_data['equations']

print("Variables:", parsed_data['variables'][0])
print("Measured Variables:", parsed_data['measured_variables'][0])
print("Constants:", parsed_data['constants'][0])
print("Non-Measured Variables:", parsed_data['non_measured_variables'][0])
print("Equations:", parsed_data['equations'][0])
print("Derivatives:", parsed_data['derivatives'][0])

Variables: ['Fc', 'Fg', 'W', 'd1', 'd2', 'm1', 'm2', 'p', 'T']
Measured Variables: ['Fc', 'Fg', 'W', 'd1', 'd2', 'm1', 'm2', 'p', 'T']
Constants: ['G', 'c']
Non-Measured Variables: ['Equations:']
Equations: ['2*c*d2xdt2*d1*m1 + c*Fc*d2 - 2*d2xdt2*d1*p', 'd2xdt2*m1 - 2*d2xdt2*m2 + Fc + Fg', 'c^2*m1 - W', '3*G*m1 - c^2*d2 + d2xdt2*d1^2 + d2xdt2*d2^2']
Derivatives: ['d2xdt2']


In [ ]:
n = len(equations)
for i in range(n):
    sorted_equations = sort_equations_by_variable_count(equations[i])
    variables = collect_variables_in_sequence_and_reverse(equations[i])
    dataset = generate_data_for_system(sorted_equations)

    data_dir = 'axiom_discovery_benchmark/axiom_system_data/'

    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    
    # Write the data to a file
    file_name = f'axiom_system_{i}_data.dat'
    file_path = os.path.join(data_dir, file_name)

    with open(file_path, 'w') as file:
        # Write the equation
        file.write("System:\n")
        for equation in equations[i]:
            file.write(equation + '\n')
        file.write('\n')

        # Write the variables
        file.write(f'Variables: {", ".join(variables)}\n')
        file.write('\n')
        file.write('Data:\n')
        # Write the data points
        for j in range(len(dataset[variables[0]])):
            values = [f'{var}={dataset[var][j]}' for var in variables]
            file.write(f'{", ".join(values)}\n')

    print(f"Data written to {file_path}")

Output from Macaulay2:
Ring defined: R
Axioms defined: {m1*d2xdt2-Fc, p*c-m2*c^2+W, G*m2-d2^2*d2xdt2+d2*c^2, -Fg*p+2*p*m2*d2xdt2+4*c*m1*Fc}
Measured variables defined: {Fg, G, d2, p, m2, W, c, m1, d2xdt2, Fc}
Non-measured variables defined: {}
Ideal defined: ideal(m1*d2xdt2-Fc,p*c-m2*c^2+W,G*m2-d2^2*d2xdt2+d2*c^2,-Fg*p+2*p*m2*d2xdt2+4*c*m1*Fc)
Gröbner basis: matrix {{m1*d2xdt2-Fc, p*c-m2*c^2+W, G*m2-d2^2*d2xdt2+d2*c^2, Fg*m2*c^2-Fg*W-2*m2^2*c^2*d2xdt2+2*m2*W*d2xdt2-4*c^2*m1*Fc, Fg*p-2*p*m2*d2xdt2-4*c*m1*Fc, Fg*G*W-Fg*d2^2*c^2*d2xdt2+Fg*d2*c^4+4*G*c^2*m1*Fc+2*d2^2*m2*c^2*d2xdt2^2-2*d2^2*W*d2xdt2^2-2*d2*m2*c^4*d2xdt2+2*d2*W*c^2*d2xdt2}}
Ring of Measured Variables defined: Rproj
Eliminated Ideal: ideal(m1*d2xdt2-Fc,p*c-m2*c^2+W,G*m2-d2^2*d2xdt2+d2*c^2,-Fg*p+2*p*m2*d2xdt2+4*c*m1*Fc)
Grobner basis of the eliminated ideal: matrix {{m1*d2xdt2-Fc, p*c-m2*c^2+W, G*m2-d2^2*d2xdt2+d2*c^2, Fg*m2*c^2-Fg*W-2*m2^2*c^2*d2xdt2+2*m2*W*d2xdt2-4*c^2*m1*Fc, Fg*p-2*p*m2*d2xdt2-4*c*m1*Fc, Fg*G*W-Fg*d2^2*c^2*